In [14]:
import os
import requests
from bs4 import BeautifulSoup
import re
import csv
import pandas as pd
from newsapi import NewsApiClient
from urllib.parse import urlparse, parse_qs
from openpyxl import load_workbook
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')
nltk.download('sentiwordnet')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.tag import pos_tag

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer

from readability.exceptions import ReadabilityException

import stanza
stanza.download('en')  # Download the English model

from readability import Readability

import spacy
nlp_spacy = spacy.load("en_core_web_sm")

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel

import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis
from gensim.models import Phrases
from gensim.models.phrases import Phraser

import statistics
import numpy as np
import pandas as pd
from IPython.display import display

import csv
from tabulate import tabulate
import newspaper

import torch
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

import os
import requests
from bs4 import BeautifulSoup
import re

from openpyxl import Workbook
import openpyxl

[nltk_data] Downloading package punkt to /home/pierluigi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pierluigi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/pierluigi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/pierluigi/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /home/pierluigi/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/pierluigi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


2023-06-05 17:16:11 INFO: Downloading default packages for language: en (English) ...
2023-06-05 17:16:12 INFO: File exists: /home/pierluigi/stanza_resources/en/default.zip
2023-06-05 17:16:19 INFO: Finished downloading models and saved to /home/pierluigi/stanza_resources.


In [15]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,sentiment', tokenize_no_ssplit=False, max_split_size_mb=16, use_gpu=False)

2023-06-05 17:16:20 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-06-05 17:16:21 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| sentiment | sstplus  |

2023-06-05 17:16:21 INFO: Using device: cpu
2023-06-05 17:16:21 INFO: Loading: tokenize
2023-06-05 17:16:21 INFO: Loading: sentiment
2023-06-05 17:16:22 INFO: Done loading processors!


In [16]:
# Load the MPQA lexicon
lexicon = pd.read_csv("/home/pierluigi/Documents/echo_chambers_intership/Code analysis/NLP/Single modules/subjclueslen1-HLTEMNLP05.tff", sep=" ", header=None, 
                      names=["type", "len", "word", "pos", "stemmed", "polarity", "strength"])

lexicon["type"] = lexicon["type"].str[5:]
lexicon["word"] = lexicon["word"].str[len("word1="):]
lexicon["polarity"] = lexicon["polarity"].str[len("priorpolarity="):]
cols_to_remove = ["len", "pos", "stemmed", "strength"]
lexicon = lexicon.drop(columns=cols_to_remove)
lexicon["type"] = lexicon["type"].replace("weaksubj", 1)
lexicon["type"] = lexicon["type"].replace("strongsubj", 2)
lexicon["polarity"] = lexicon["polarity"].replace("negative", -1)
lexicon["polarity"] = lexicon["polarity"].replace("positive", 1)
lexicon["polarity"] = lexicon["polarity"].replace("both", 0)
lexicon["polarity"] = lexicon["polarity"].replace("neutral", 0)

In [17]:
def save_article_text(url):
    # Set headers to mimic a web browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    # Send a GET request to the URL with headers
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract the title, subtitle, description, and main text
    title_element = soup.find('title')
    title = title_element.text.strip() if title_element else ""

    subtitle_element = soup.find('meta', attrs={'name': 'description'})
    subtitle = subtitle_element['content'].strip() if subtitle_element and 'content' in subtitle_element.attrs else ""

    description_element = soup.find('meta', attrs={'name': 'og:description'})
    description = description_element['content'].strip() if description_element and 'content' in description_element.attrs else ""

    # Find and exclude unwanted elements by class names or content patterns
    unwanted_elements = soup.find_all(['script', 'style', 'a', 'div', 'span'], class_=['follow-us', 'newsletter', 'advertisement'])
    patterns_to_exclude = ['next article', 'read next', 'correlated']
    for element in unwanted_elements:
        if any(pattern in str(element).lower() for pattern in patterns_to_exclude):
            element.extract()

    # Find and exclude footer container and "All rights reserved" text
    footer_elements = soup.find_all(['footer', 'div'], class_=['footer', 'bottom-footer'])
    for element in footer_elements:
        element.extract()
    all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
    for element in all_rights_reserved_elements:
        element.extract()

    # Find the main text element(s) based on the HTML structure of the page
    main_text_elements = soup.find_all('p')
    main_text = "\n\n".join([element.text.strip() for element in main_text_elements if element.text.strip()])

    # Set the subtitle to the description if it is empty
    if not subtitle:
        subtitle = description.strip()

    # Concatenate the extracted strings
    article_text = f"{title}\n\n{subtitle}\n\n{main_text}"

    return article_text

In [18]:
def extract_real_url(google_news_url):
    response = requests.get(google_news_url, cookies = {'CONSENT' : 'YES+'})
    real_url = response.url
    return real_url

In [19]:
def process_data(data_dict):
    processed_data = {}
    
    for sheet_name, data_list in data_dict.items():
        sheet_data = {}
        
        for index, url in enumerate(data_list, start=1):
            try:
                article_url = extract_real_url(url)
                article_text = save_article_text(article_url)
                key = f"article{index}"
                sheet_data[key] = article_text
            except (requests.exceptions.RequestException, requests.exceptions.HTTPError) as e:
                # Handle the exception and continue with the next URL
                print(f"Error processing URL: {url}")
                print(f"Error message: {str(e)}")
                continue
        
        processed_data[sheet_name] = sheet_data
    
    return processed_data

In [20]:
def get_flesch_grading_levels_readability_scores(data_dict):
    readability_data = {}  # Dictionary to store sentiment scores for each article
    
    for sheet_name, sheet_data in data_dict.items():
        sheet_readability = []
        
        for article_key, article_text in sheet_data.items():
            word_count = len(article_text.split())

            if word_count < 100:
                # Skip articles with less than 100 words
                continue

            try:
                r = Readability(article_text)
                f = r.flesch()
                score = f.score
                
                if score >= 0 and score <= 29:
                    grade = {'grade': 'F6'}
                elif score >= 30 and score <= 49:
                    grade = {'grade': 'F5'}
                elif score >= 50 and score <= 59:
                    grade = {'grade': 'F4'}
                elif score >= 60 and score <= 69:
                    grade = {'grade': 'F3'}
                elif score >= 70 and score <= 79:
                    grade = {'grade': 'F2'}
                elif score >= 80 and score <= 89:
                    grade = {'grade': 'F1'}
                else:
                    grade = {'grade': 'F0'}
                
                readability_data[(sheet_name, article_key)] = grade
                
            except ReadabilityException:
                # Skip articles that do not meet the word count requirement
                continue

        readability_data[sheet_name] = sheet_readability
    
    return readability_data



In [21]:
def save_readability_scores_to_excel_flesch_reading_ease(processed_data_list, readability_scores_list, output_file_path):
    output_workbook = openpyxl.Workbook()
    
    for i, processed_data in enumerate(processed_data_list):
        readability_scores = readability_scores_list[i]
        sheet_name = f"Sheet {i+1}"
        
        if i == 0:
            output_sheet = output_workbook.active
            output_sheet.title = sheet_name
        else:
            output_sheet = output_workbook.create_sheet(title=sheet_name)
        
        output_sheet['A1'] = 'Date'
        
        row = 2
        
        for sheet_name, sheet_data in processed_data.items():
            output_sheet.cell(row=row, column=1, value=sheet_name)
            column = 2
            
            for article_index, (article_key, article_text) in enumerate(sheet_data.items()):
                if (sheet_name, article_key) in readability_scores:
                    scores = readability_scores[(sheet_name, article_key)]
                    output_sheet.cell(row=1, column=column, value=f"Flesch Reading Ease grade {article_index+1}")
                    output_sheet.cell(row=row, column=column, value=scores['grade'])
                else:
                    output_sheet.cell(row=1, column=column, value=f"Flesch Reading Ease grade {article_index+1}")
                    output_sheet.cell(row=row, column=column, value='N/A')
                
                column += 1
            row += 1
    
    output_workbook.save(output_file_path)


In [22]:
def get_dale_chall_levels_readability_scores(data_dict):
    readability_data = {}  # Dictionary to store sentiment scores for each article
    
    for sheet_name, sheet_data in data_dict.items():
        sheet_readability = []
        
        for article_key, article_text in sheet_data.items():
            word_count = len(article_text.split())

            if word_count < 100:
                # Skip articles with less than 100 words
                continue

            try:
                r = Readability(article_text)
                dc = r.dale_chall()
                score = dc.score
                
                if score >= 0 and score <= 4.9:
                    grade = {'grade': 'D0'}
                elif score >= 5.0 and score <= 5.9:
                    grade = {'grade': 'D1'}
                elif score >= 6.0 and score <= 6.9:
                    grade = {'grade': 'D2'}
                elif score >= 7.0 and score <= 7.9:
                    grade = {'grade': 'D3'}
                elif score >= 8.0 and score <= 8.9:
                    grade = {'grade': 'D4'}
                elif score >= 9.0 and score <= 9.9:
                    grade = {'grade': 'D5'}
                else:
                    grade = {'grade': 'D6'}
                
                readability_data[(sheet_name, article_key)] = grade
                
            except ReadabilityException:
                # Skip articles that do not meet the word count requirement
                continue

        readability_data[sheet_name] = sheet_readability
    
    return readability_data



In [23]:
def save_readability_scores_to_excel_dale_chall(processed_data_list, readability_scores_list, output_file_path):
    output_workbook = openpyxl.load_workbook(output_file_path)
    
    for i, processed_data in enumerate(processed_data_list):
        readability_scores = readability_scores_list[i]
        sheet_name = f"Sheet {i+1}"
        
        # Check if the sheet already exists in the workbook
        if sheet_name in output_workbook.sheetnames:
            output_sheet = output_workbook[sheet_name]
        else:
            raise ValueError(f"Sheet '{sheet_name}' does not exist in the workbook.")
        
        row = 2
        
        for sheet_name, sheet_data in processed_data.items():
            output_sheet.cell(row=row, column=1, value=sheet_name)
            column = 12
            
            for article_index, (article_key, article_text) in enumerate(sheet_data.items()):
                if (sheet_name, article_key) in readability_scores:
                    scores = readability_scores[(sheet_name, article_key)]
                    output_sheet.cell(row=1, column=column, value=f"Dale Chall grade {article_index+1}")
                    output_sheet.cell(row=row, column=column, value=scores['grade'])
                else:
                    output_sheet.cell(row=1, column=column, value=f"Dale Chall grade {article_index+1}")
                    output_sheet.cell(row=row, column=column, value='N/A')
                
                column += 1
            row += 1
    
    output_workbook.save(output_file_path)


In [24]:
def excel_to_csv(excel_file):
    # Load the Excel file
    xls = pd.ExcelFile(excel_file)

    # Get the sheet names
    sheet_names = xls.sheet_names

    # Iterate over each sheet and convert to CSV
    for sheet_name in sheet_names:
        # Read the sheet as a DataFrame
        df = pd.read_excel(excel_file, sheet_name=sheet_name)

        # Generate the CSV file name
        csv_file = f"{sheet_name}.csv"

        # Save the DataFrame as a CSV file
        df.to_csv(csv_file, index=False)

        print(f"CSV file '{csv_file}' created for sheet '{sheet_name}'")

In [25]:
def main():

    workbook = load_workbook('/home/pierluigi/Documents/echo_chambers_intership/Code analysis/NLP/Analysis/Text analytics/Text Analytics.xlsx')

    data_U1 = {}  # Dictionary to store data from range 1 - U1
    data_U2 = {}  # Dictionary to store data from range 2 - U2
    data_U3 = {}  # Dictionary to store data from range 3 - U3
    data_U4 = {}  # Dictionary to store data from range 4 - U4

    # Iterate over each sheet in the workbook
    for sheet_name in workbook.sheetnames:
        worksheet = workbook[sheet_name]

        # Specify the range of cells for data_U1
        range1_start_cell = 'C2'  # Replace with the starting cell of range 1
        range1_end_cell = 'L2'  # Replace with the ending cell of range 1
        
        data_list1 = []
        
        # Iterate over the cells within range 1
        for row in worksheet[range1_start_cell:range1_end_cell]:
            for cell in row:
                data_list1.append(cell.value)
        
        data_U1[sheet_name] = data_list1

        # Specify the range of cells for data_U2
        range2_start_cell = 'C3'  # Replace with the starting cell of range 2
        range2_end_cell = 'L3'  # Replace with the ending cell of range 2
        
        data_list2 = []
        
        # Iterate over the cells within range 2
        for row in worksheet[range2_start_cell:range2_end_cell]:
            for cell in row:
                data_list2.append(cell.value)
        
        data_U2[sheet_name] = data_list2

        # Specify the range of cells for data_U3
        range3_start_cell = 'C4'  # Replace with the starting cell of range 3
        range3_end_cell = 'L4'  # Replace with the ending cell of range 3
        
        data_list3 = []
        
        # Iterate over the cells within range 3
        for row in worksheet[range3_start_cell:range3_end_cell]:
            for cell in row:
                data_list3.append(cell.value)
        
        data_U3[sheet_name] = data_list3

        # Specify the range of cells for data_U4
        range4_start_cell = 'C5'  # Replace with the starting cell of range 4
        range4_end_cell = 'L5'  # Replace with the ending cell of range 4
        
        data_list4 = []
        
        # Iterate over the cells within range 4
        for row in worksheet[range4_start_cell:range4_end_cell]:
            for cell in row:
                data_list4.append(cell.value)
        
        data_U4[sheet_name] = data_list4

    workbook.close()

    # Process the data
    processed_data1 = process_data(data_U1)
    processed_data2 = process_data(data_U2)
    processed_data3 = process_data(data_U3)
    processed_data4 = process_data(data_U4)

    # Flesch Reading Ease readability scores
    flesch_reading_ease_readability_scores1 = get_flesch_grading_levels_readability_scores(processed_data1)
    flesch_reading_ease_readability_scores2 = get_flesch_grading_levels_readability_scores(processed_data2)
    flesch_reading_ease_readability_scores3= get_flesch_grading_levels_readability_scores(processed_data3)
    flesch_reading_ease_readability_scores4 = get_flesch_grading_levels_readability_scores(processed_data4)

    processed_data_list = [processed_data1, processed_data2, processed_data3, processed_data4]
    flesch_reading_ease_scores_list = [flesch_reading_ease_readability_scores1, flesch_reading_ease_readability_scores2, flesch_reading_ease_readability_scores3, flesch_reading_ease_readability_scores4]
    
    output_file_path = '/home/pierluigi/Desktop/readability.xlsx'

    save_readability_scores_to_excel_flesch_reading_ease(processed_data_list, flesch_reading_ease_scores_list, output_file_path)

    # Dale Chall readability scores
    dale_chall_readability_scores1 = get_dale_chall_levels_readability_scores(processed_data1)
    dale_chall_readability_scores2 = get_dale_chall_levels_readability_scores(processed_data2)
    dale_chall_readability_scores3 = get_dale_chall_levels_readability_scores(processed_data3)
    dale_chall_readability_scores4 = get_dale_chall_levels_readability_scores(processed_data4)

    dale_chall_scores_list = [dale_chall_readability_scores1, dale_chall_readability_scores2, dale_chall_readability_scores3, dale_chall_readability_scores4]

    save_readability_scores_to_excel_dale_chall(processed_data_list, dale_chall_scores_list, output_file_path)


    

In [26]:
# Run the main function
if __name__ == "__main__":
    main()

/tmp/ipykernel_21844/2363556993.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
/tmp/ipykernel_21844/2363556993.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
/tmp/ipykernel_21844/2363556993.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
/tmp/ipykernel_21844/2363556993.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
/tmp/ipykernel_2

Error processing URL: https://news.google.com/articles/CBMiSGh0dHBzOi8vd3d3LmxpZmVzdHlsZWFzaWEuY29tL2tsL3N0eWxlL2Zhc2hpb24vbWV0LWdhbGEtMjAyMy1iZXN0LW1lbWVzL9IBTGh0dHBzOi8vd3d3LmxpZmVzdHlsZWFzaWEuY29tL2tsL3N0eWxlL2Zhc2hpb24vbWV0LWdhbGEtMjAyMy1iZXN0LW1lbWVzL2FtcC8?hl=en-US&gl=US&ceid=US%3Aen
Error message: 403 Client Error: Forbidden for url: https://www.lifestyleasia.com/kl/style/fashion/met-gala-2023-best-memes/


/tmp/ipykernel_21844/2363556993.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
/tmp/ipykernel_21844/2363556993.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
/tmp/ipykernel_21844/2363556993.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
/tmp/ipykernel_21844/2363556993.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
/tmp/ipykernel_2

Error processing URL: https://news.google.com/articles/CBMiUmh0dHBzOi8vc2NyZWVucmFudC5jb20vdmFuZGVycHVtcC1ydWxlcy10b20tc2FuZG92YWwtZGlzZGFpbi1rYXRpZS1kZWVwZXItbWVhbmluZy_SAQA?hl=en-US&gl=US&ceid=US%3Aen
Error message: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


/tmp/ipykernel_21844/2363556993.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
/tmp/ipykernel_21844/2363556993.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
/tmp/ipykernel_21844/2363556993.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
/tmp/ipykernel_21844/2363556993.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
/tmp/ipykernel_2

Error processing URL: https://news.google.com/articles/CBMiUWh0dHBzOi8vbmV3cy55YWhvby5jb20vc2VuYXRlLXJlcHVibGljYW5zLWluY2x1ZGluZy1tY2Nvbm5lbGwtZGVidC0yMjAwMzg1ODYuaHRtbNIBWWh0dHBzOi8vbmV3cy55YWhvby5jb20vYW1waHRtbC9zZW5hdGUtcmVwdWJsaWNhbnMtaW5jbHVkaW5nLW1jY29ubmVsbC1kZWJ0LTIyMDAzODU4Ni5odG1s?hl=en-US&gl=US&ceid=US%3Aen
Error message: 404 Client Error: Not Found for url: https://news.yahoo.com/senate-republicans-including-mcconnell-debt-220038586.html


/tmp/ipykernel_21844/2363556993.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
/tmp/ipykernel_21844/2363556993.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
/tmp/ipykernel_21844/2363556993.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
/tmp/ipykernel_21844/2363556993.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
/tmp/ipykernel_2

Error processing URL: https://news.google.com/articles/CBMiamh0dHBzOi8vd3d3LndhdnkuY29tL25ld3MvbWlsaXRhcnkva2lnZ2Fucy1kaXNwdXRlcy1jbGFpbXMtdGhhdC1idWRnZXQtcGxhbi13b3VsZC1zbGFzaC12ZXRlcmFucy1iZW5lZml0cy_SAW5odHRwczovL3d3dy53YXZ5LmNvbS9uZXdzL21pbGl0YXJ5L2tpZ2dhbnMtZGlzcHV0ZXMtY2xhaW1zLXRoYXQtYnVkZ2V0LXBsYW4td291bGQtc2xhc2gtdmV0ZXJhbnMtYmVuZWZpdHMvYW1wLw?hl=en-US&gl=US&ceid=US%3Aen
Error message: 451 Client Error: Unavailable For Legal Reasons for url: https://www.wavy.com/news/military/kiggans-disputes-claims-that-budget-plan-would-slash-veterans-benefits/


/tmp/ipykernel_21844/2363556993.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
/tmp/ipykernel_21844/2363556993.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
/tmp/ipykernel_21844/2363556993.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
/tmp/ipykernel_21844/2363556993.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
/tmp/ipykernel_2